In [1]:
from DCC import *
from Utils import *
from Plots import *
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pathlib import Path
from collections import defaultdict

In [2]:
init_plotting()

In [3]:
dataset2name = {
    "Bala_classification_dataset.csv": "Bala Classification",
    "Bala_regression_dataset.csv": "Bala Regression",
    "bandgap.csv": "Bandgap",
    "BMDS_data.csv": "BMDS",
    "Crystal_structure.csv": "Crystal Structure",
    "Glass.csv": "Glass",
    "PUE.csv": "PUE",
}

from enum import Enum


class PerturbationType(Enum):
    Deletion = "Deletion"
    AdditionLinear = "AdditionLinear"
    AdditionRand = "AdditionRand"
    ReplacementLinear = "ReplacementLinear"
    ReplacementRand = "ReplacementRand"

In [4]:
from sklearn.datasets import make_classification


def generate_random_according_df(
    n: int, m: int, df: pd.DataFrame, task_type=None, target_col=None
):
    cols = df.columns
    new_data = []
    for i in range(n):
        new_row = []
        for j in range(m):
            new_row.append(random.uniform(df[cols[j]].min(), df[cols[j]].max()))
        new_data.append(new_row)
    new_df = pd.DataFrame(new_data, columns=cols)
    if task_type == "classification":
        if target_col is not None:
            new_df[target_col] = np.random.choice(
                df[target_col].unique(), size=n, replace=True
            )
    return new_df


def generate_linear_according_df(
    n: int, m: int, df: pd.DataFrame, task_type=None, target_col=None
):
    if task_type == "classification":
        # Generate a classification dataset
        X, y = make_classification(
            n_samples=n,
            n_features=m - 1,
            n_classes=len(df[target_col].unique()),
            n_informative=(m - 1) // 2,
        )[:2]
    else:
        X, y = make_regression(
            n_samples=n,
            n_features=m - 1,
            noise=0.1,
        )[:2]

    X_cols = [x for x in df.columns if x != target_col]
    df_linear = pd.DataFrame(X, columns=X_cols)
    df_linear[target_col] = y
    # normalize the data to the range of df
    for col in df.columns:
        min_val = df[col].min()
        max_val = df[col].max()
        df_linear[col] = (df_linear[col] - df_linear[col].min()) / (
            df_linear[col].max() - df_linear[col].min()
        ) * (max_val - min_val) + min_val

    return df_linear


def dcc_exp(
    data: pd.DataFrame,
    ptb_type: PerturbationType,
    task_type,
    target_col,
    ratio=0.1,
):
    n, m = data.shape
    if ptb_type == PerturbationType.Deletion:
        perturbed_data = data.sample(frac=1 - ratio)
    elif ptb_type == PerturbationType.AdditionLinear:
        df_linear = generate_linear_according_df(
            int(n * ratio), m, data, task_type, target_col
        )
        perturbed_data = pd.concat([data, df_linear], ignore_index=True)
    elif ptb_type == PerturbationType.AdditionRand:
        df_rand = generate_random_according_df(
            int(n * ratio), m, data, task_type, target_col
        )
        perturbed_data = pd.concat([data, df_rand], ignore_index=True)
    elif ptb_type == PerturbationType.ReplacementLinear:
        df_linear = generate_linear_according_df(
            int(n * ratio), m, data, task_type, target_col
        )
        perturbed_data = random_replace_rows(data, df_linear)
    elif ptb_type == PerturbationType.ReplacementRand:
        df_rand = generate_random_according_df(
            int(n * ratio), m, data, task_type, target_col
        )
        perturbed_data = random_replace_rows(data, df_rand)
    else:
        raise ValueError(f"Unknown perturbation type: {ptb_type}")

    return perturbed_data

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import r2_score, accuracy_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from joblib import Parallel, delayed

dataset_config = {
    "Bala_classification_dataset.csv": {
        "target_col": "Formability",
        "type": "classification",
    },
    "Bala_regression_dataset.csv": {
        "target_col": "Ferroelectric_Tc_in_Kelvin",
        "type": "regression",
    },
    "bandgap.csv": {
        "target_col": "target",
        "type": "regression",
    },
    "BMDS_data.csv": {
        "target_col": "soc_bandgap",
        "type": "regression",
    },
    "Crystal_structure.csv": {
        "target_col": "Lowest distortion",
        "type": "classification",
    },
    "Glass.csv": {
        "target_col": "Type of glass",
        "type": "classification",
    },
    "PUE.csv": {
        "target_col": "logYM",
        "type": "regression",
    },
}


def get_scores_single(df, target_col, type, repeat):
    X = df.drop(columns=[target_col])
    y = df[target_col]
    cur_scores = []
    n_splits = 5
    if type == "classification":
        kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=repeat)
    else:
        kf = KFold(n_splits=n_splits, shuffle=True, random_state=repeat)
    for train_index, test_index in kf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        if type == "regression":
            model = RandomForestRegressor(n_estimators=500, random_state=42)
        elif type == "classification":
            model = RandomForestClassifier(n_estimators=500, random_state=42)
        else:
            raise ValueError("type must be either 'regression' or 'classification'")
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        if type == "regression":
            cur_scores.append(r2_score(y_test, y_pred))
        elif type == "classification":
            cur_scores.append(accuracy_score(y_test, y_pred))
    return cur_scores


def get_scores(df, target_col, type):
    repeats = 10

    all_scores = Parallel(n_jobs=-1)(
        delayed(get_scores_single)(df, target_col, type, i) for i in range(repeats)
    )
    all_scores = np.array(all_scores)
    return all_scores

In [6]:
import pandas as pd
df = pd.read_csv("processed_data/PUE.csv")

In [7]:
def corr_max(df, target_col, corr_func):
    corr = corr_func(df)
    return corr[target_col].abs().sort_values(ascending=False).iloc[1:].max()

def corr_cnt(df, target_col, corr_func):
    corr = corr_func(df)
    corr = corr[target_col].abs().sort_values(ascending=False).iloc[1:]
    # get corr > 0.5 ratio
    good_feature_ratio = float((corr > 0.5).sum()) / (len(corr) - 1)
    normalized_corr = float(((0.1 < corr) & (corr <= 0.5)).sum()) / (len(corr) - 1)
    bad_feature_ratio = float((corr <= 0.1).sum()) / (len(corr) - 1)
    return good_feature_ratio, normalized_corr, bad_feature_ratio

In [8]:
ptb_ratios = [0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15]

for ptb_ratio in ptb_ratios:
    results = defaultdict(dict)
    dataset_dir = Path("processed_data")
    for d in dataset_dir.glob("*.csv"):
        df = pd.read_csv(d)
        target_col = dataset_config[d.name]["target_col"]
        task_type = dataset_config[d.name]["type"]
        ori_scores = get_scores(df, target_col, task_type)
        results[d.name]["ori_score"] = ori_scores
        results[d.name]["name"] = dataset2name[d.name]
        results[d.name]["type"] = task_type
        results[d.name]["target_col"] = target_col

        results[d.name][PerturbationType.Deletion.value] = {}
        results[d.name][PerturbationType.AdditionLinear.value] = {}
        results[d.name][PerturbationType.AdditionRand.value] = {}
        results[d.name][PerturbationType.ReplacementLinear.value] = {}
        results[d.name][PerturbationType.ReplacementRand.value] = {}

        print(task_type, target_col)
        # Deletion
        for ptb_type in PerturbationType:
            print(f"Perturbation type: {ptb_type.value}")
            perturbed_df = dcc_exp(df, ptb_type, task_type, target_col, ratio=ptb_ratio)
            dcc_pearson = dcc_diff(df, perturbed_df, eps=0.04, corr_func=pearson_matrix)
            dcc_spearman = dcc_diff(
                df, perturbed_df, eps=0.04, corr_func=spearman_matrix
            )
            dcc_kendall = dcc_diff(df, perturbed_df, eps=0.04, corr_func=kendall_matrix)
            dcc_mi = dcc_diff(df, perturbed_df, eps=0.04, corr_func=mutual_info_matrix)
            dcc_js = dcc_diff(df, perturbed_df, eps=0.04, corr_func=js_corr_matrix)
            dcc_wd = dcc_diff(df, perturbed_df, eps=0.04, corr_func=wd_corr_matrix)
            dcc_xi = dcc_diff(df, perturbed_df, eps=0.04, corr_func=xi_matrix)
            dcc_dcor = dcc_diff(df, perturbed_df, eps=0.04, corr_func=dcor_matrix)
            # dcc_hsic = dcc_diff(df, perturbed_df, eps=0.04, corr_func=nhsic_matrix)

            scores = get_scores(perturbed_df, target_col, task_type)
            results[d.name][ptb_type.value]["dcc_pearson"] = dcc_pearson
            results[d.name][ptb_type.value]["dcc_spearman"] = dcc_spearman
            results[d.name][ptb_type.value]["dcc_kendall"] = dcc_kendall
            results[d.name][ptb_type.value]["dcc_mi"] = dcc_mi
            results[d.name][ptb_type.value]["dcc_js"] = dcc_js
            results[d.name][ptb_type.value]["dcc_wd"] = dcc_wd
            results[d.name][ptb_type.value]["dcc_xi"] = dcc_xi
            results[d.name][ptb_type.value]["dcc_dcor"] = dcc_dcor
            # results[d.name][ptb_type.value]["dcc_hsic"] = dcc_hsic

            # dcc_diff_pearson = dcc_difference(
            #     df, perturbed_df, corr_func=pearson_matrix
            # )
            # dcc_diff_spearman = dcc_difference(
            #     df, perturbed_df, corr_func=spearman_matrix
            # )
            # dcc_diff_kendall = dcc_difference(
            #     df, perturbed_df, corr_func=kendall_matrix
            # )
            # dcc_diff_mi = dcc_difference(df, perturbed_df, corr_func=mutual_info_matrix)
            # dcc_diff_js = dcc_difference(df, perturbed_df, corr_func=js_corr_matrix)
            # dcc_diff_wd = dcc_difference(df, perturbed_df, corr_func=wd_corr_matrix)
            # results[d.name][ptb_type.value]["dcc_diff_pearson"] = dcc_diff_pearson
            # results[d.name][ptb_type.value]["dcc_diff_spearman"] = dcc_diff_spearman
            # results[d.name][ptb_type.value]["dcc_diff_kendall"] = dcc_diff_kendall
            # results[d.name][ptb_type.value]["dcc_diff_mi"] = dcc_diff_mi
            # results[d.name][ptb_type.value]["dcc_diff_js"] = dcc_diff_js
            # results[d.name][ptb_type.value]["dcc_diff_wd"] = dcc_diff_wd


            results[d.name][ptb_type.value]["corr_max_pearson"] = corr_max(
                perturbed_df, target_col, pearson_matrix
            )
            results[d.name][ptb_type.value]["corr_max_spearman"] = corr_max(
                perturbed_df, target_col, spearman_matrix
            )
            results[d.name][ptb_type.value]["corr_max_kendall"] = corr_max(
                perturbed_df, target_col, kendall_matrix
            )
            results[d.name][ptb_type.value]["corr_max_mi"] = corr_max(
                perturbed_df, target_col, mutual_info_matrix
            )
            results[d.name][ptb_type.value]["corr_max_js"] = corr_max(
                perturbed_df, target_col, js_corr_matrix
            )
            results[d.name][ptb_type.value]["corr_max_wd"] = corr_max(
                perturbed_df, target_col, wd_corr_matrix
            )
            results[d.name][ptb_type.value]["corr_max_xi"] = corr_max(
                perturbed_df, target_col, xi_matrix
            )
            results[d.name][ptb_type.value]["corr_max_dcor"] = corr_max(
                perturbed_df, target_col, dcor_matrix
            )
            # results[d.name][ptb_type.value]["corr_max_hsic"] = corr_max(
            #     perturbed_df, target_col, nhsic_matrix
            # )


            (results[d.name][ptb_type.value]["corr_cnt_good_pearson"], _, _) = corr_cnt(
                perturbed_df, target_col, pearson_matrix
            )
            (results[d.name][ptb_type.value]["corr_cnt_good_spearman"], _, _) = (
                corr_cnt(perturbed_df, target_col, spearman_matrix)
            )
            (results[d.name][ptb_type.value]["corr_cnt_good_kendall"], _, _) = corr_cnt(
                perturbed_df, target_col, kendall_matrix
            )
            (results[d.name][ptb_type.value]["corr_cnt_good_mi"], _, _) = corr_cnt(
                perturbed_df, target_col, mutual_info_matrix
            )
            (results[d.name][ptb_type.value]["corr_cnt_good_js"], _, _) = corr_cnt(
                perturbed_df, target_col, js_corr_matrix
            )
            (results[d.name][ptb_type.value]["corr_cnt_good_wd"], _, _) = corr_cnt(
                perturbed_df, target_col, wd_corr_matrix
            )
            (results[d.name][ptb_type.value]["corr_cnt_good_xi"], _, _) = corr_cnt(
                perturbed_df, target_col, xi_matrix
            )
            (results[d.name][ptb_type.value]["corr_cnt_good_dcor"], _, _) = corr_cnt(
                perturbed_df, target_col, dcor_matrix
            )
            # (results[d.name][ptb_type.value]["corr_cnt_good_hsic"], _, _) = corr_cnt(
            #     perturbed_df, target_col, nhsic_matrix
            # )

            results[d.name][ptb_type.value]["scores"] = scores

    import pickle
    from pathlib import Path

    with open(f"results/dcc-{ptb_ratio}.pkl", "wb") as f:
        pickle.dump(results, f)
    print(
        f"Finished processing with perturbation ratio {ptb_ratio}. Results saved to results/dcc_results-{ptb_ratio}.pkl"
    )

classification Formability
Perturbation type: Deletion
Perturbation type: AdditionLinear
Perturbation type: AdditionRand
Perturbation type: ReplacementLinear
Perturbation type: ReplacementRand
regression Ferroelectric_Tc_in_Kelvin
Perturbation type: Deletion
Perturbation type: AdditionLinear
Perturbation type: AdditionRand
Perturbation type: ReplacementLinear
Perturbation type: ReplacementRand
regression target
Perturbation type: Deletion
Perturbation type: AdditionLinear
Perturbation type: AdditionRand
Perturbation type: ReplacementLinear
Perturbation type: ReplacementRand
regression soc_bandgap
Perturbation type: Deletion
Perturbation type: AdditionLinear
Perturbation type: AdditionRand
Perturbation type: ReplacementLinear
Perturbation type: ReplacementRand
classification Lowest distortion
Perturbation type: Deletion
Perturbation type: AdditionLinear
Perturbation type: AdditionRand
Perturbation type: ReplacementLinear
Perturbation type: ReplacementRand
classification Type of glass
Pe

In [9]:
import os
os.listdir("results")

['dcc-0.05.pkl',
 'dcc-0.06.pkl',
 'dcc-0.07.pkl',
 'dcc-0.08.pkl',
 'dcc-0.09.pkl',
 'dcc-0.1.pkl',
 'dcc-0.11.pkl',
 'dcc-0.12.pkl',
 'dcc-0.13.pkl',
 'dcc-0.14.pkl',
 'dcc-0.15.pkl',
 'dcc-all.pkl',
 'dcc-vs-scores.pkl',
 'dcc_fi_results-dcor.pkl',
 'dcc_fi_results-JS.pkl',
 'dcc_fi_results-k.pkl',
 'dcc_fi_results-MI.pkl',
 'dcc_fi_results-p.pkl',
 'dcc_fi_results-s.pkl',
 'dcc_fi_results-Xi.pkl',
 'dcc_results-dcor.pkl',
 'dcc_results-JS.pkl',
 'dcc_results-KD.pkl',
 'dcc_results-MI.pkl',
 'dcc_results-sp.pkl',
 'dcc_results-WD.pkl',
 'dcc_results-Xi.pkl',
 'dcc_results.csv',
 'dcc_results.pkl',
 'dcc_vs_theta_results-JS.pkl',
 'dcc_vs_theta_results-kendall.pkl',
 'dcc_vs_theta_results-MI.pkl',
 'dcc_vs_theta_results-spearman.pkl',
 'dcc_vs_theta_results-WD.pkl',
 'dcc_vs_theta_results.pkl',
 'DQ-results.xlsx',
 'results_0.05_dcor_matrix.pkl',
 'results_0.05_js_corr_matrix.pkl',
 'results_0.05_kendall_matrix.pkl',
 'results_0.05_mutual_info_matrix.pkl',
 'results_0.05_pearson_matr

In [10]:
data_rows = []
import pickle

for ptb_ratio in ptb_ratios:
    results = {}
    with open(f"results/dcc-{ptb_ratio}.pkl", "rb") as f:
        results = pickle.load(f)
    for dataset, r in results.items():
        for ptb_type in PerturbationType:
            row = {}
            row["type"] = r["type"]
            row["original_scores"] = r["ori_score"].mean(axis=0).mean()
            row["original_scores_std"] = r["ori_score"].std(axis=0).mean()
            if ptb_type.value in r:
                row["scores"] = r[ptb_type.value]["scores"].mean(axis=0).mean()
                row["scores_std"] = r[ptb_type.value]["scores"].std(axis=0).mean()
                
                row["dcc_pearson"] = r[ptb_type.value]["dcc_pearson"]
                row["dcc_spearman"] = r[ptb_type.value]["dcc_spearman"]
                row["dcc_kendall"] = r[ptb_type.value]["dcc_kendall"]
                row["dcc_mi"] = r[ptb_type.value]["dcc_mi"]
                row["dcc_js"] = r[ptb_type.value]["dcc_js"]
                row["dcc_wd"] = r[ptb_type.value]["dcc_wd"]
                row["dcc_xi"] = r[ptb_type.value]["dcc_xi"]
                row["dcc_dcor"] = r[ptb_type.value]["dcc_dcor"]
                # row["dcc_hsic"] = r[ptb_type.value]["dcc_hsic"]

                row["perturbation_type"] = ptb_type.value
                row["dataset"] = r["name"]
                row["ptb_ratio"] = ptb_ratio
                # row["dcc_diff_pearson"] = r[ptb_type.value]["dcc_diff_pearson"]
                # row["dcc_diff_spearman"] = r[ptb_type.value]["dcc_diff_spearman"]
                # row["dcc_diff_kendall"] = r[ptb_type.value]["dcc_diff_kendall"]
                # row["dcc_diff_mi"] = r[ptb_type.value]["dcc_diff_mi"]
                # row["dcc_diff_js"] = r[ptb_type.value]["dcc_diff_js"]
                # row["dcc_diff_wd"] = r[ptb_type.value]["dcc_diff_wd"]

                row["corr_max_pearson"] = r[ptb_type.value]["corr_max_pearson"]
                row["corr_max_spearman"] = r[ptb_type.value]["corr_max_spearman"]
                row["corr_max_kendall"] = r[ptb_type.value]["corr_max_kendall"]
                row["corr_max_mi"] = r[ptb_type.value]["corr_max_mi"]
                row["corr_max_js"] = r[ptb_type.value]["corr_max_js"]
                row["corr_max_wd"] = r[ptb_type.value]["corr_max_wd"]
                row["corr_max_xi"] = r[ptb_type.value]["corr_max_xi"]
                row["corr_max_dcor"] = r[ptb_type.value]["corr_max_dcor"]
                # row["corr_max_hsic"] = r[ptb_type.value]["corr_max_hsic"]
                
                # row["corr_min_pearson"] = r[ptb_type.value]["corr_min_pearson"]
                # row["corr_min_spearman"] = r[ptb_type.value]["corr_min_spearman"]
                # row["corr_min_kendall"] = r[ptb_type.value]["corr_min_kendall"]
                # row["corr_min_mi"] = r[ptb_type.value]["corr_min_mi"]
                # row["corr_min_js"] = r[ptb_type.value]["corr_min_js"]
                # row["corr_min_wd"] = r[ptb_type.value]["corr_min_wd"]

                row["corr_cnt_good_pearson"] = r[ptb_type.value]["corr_cnt_good_pearson"]
                row["corr_cnt_good_spearman"] = r[ptb_type.value]["corr_cnt_good_spearman"]
                row["corr_cnt_good_kendall"] = r[ptb_type.value]["corr_cnt_good_kendall"]
                row["corr_cnt_good_mi"] = r[ptb_type.value]["corr_cnt_good_mi"]
                row["corr_cnt_good_js"] = r[ptb_type.value]["corr_cnt_good_js"]
                row["corr_cnt_good_wd"] = r[ptb_type.value]["corr_cnt_good_wd"]
                row["corr_cnt_good_xi"] = r[ptb_type.value]["corr_cnt_good_xi"]
                row["corr_cnt_good_dcor"] = r[ptb_type.value]["corr_cnt_good_dcor"]
                # row["corr_cnt_good_hsic"] = r[ptb_type.value]["corr_cnt_good_hsic"]


            data_rows.append(row)

df = pd.DataFrame(data_rows)
df

,type,original_scores,original_scores_std,scores,scores_std,dcc_pearson,dcc_spearman,dcc_kendall,dcc_mi,dcc_js,...,corr_max_xi,corr_max_dcor,corr_cnt_good_pearson,corr_cnt_good_spearman,corr_cnt_good_kendall,corr_cnt_good_mi,corr_cnt_good_js,corr_cnt_good_wd,corr_cnt_good_xi,corr_cnt_good_dcor
0,classification,0.892254,0.050522,0.900000,0.041102,0.978022,1.000000,1.000000,1.000000,1.000000,...,0.362232,0.425279,0.000000,0.0,0.0,0.0,0.416667,0.583333,0.000000,0.000000
1,classification,0.892254,0.050522,0.872085,0.043611,0.560440,0.692308,0.802198,0.857143,0.846154,...,0.491441,0.384753,0.000000,0.0,0.0,0.0,0.500000,0.666667,0.000000,0.000000
2,classification,0.892254,0.050522,0.897085,0.042266,0.582418,0.626374,0.747253,0.791209,0.736264,...,0.508087,0.404915,0.000000,0.0,0.0,0.0,0.416667,0.750000,0.083333,0.000000
3,classification,0.892254,0.050522,0.881350,0.053609,0.472527,0.670330,0.769231,0.890110,0.692308,...,0.482904,0.425134,0.000000,0.0,0.0,0.0,0.416667,0.666667,0.000000,0.000000
4,classification,0.892254,0.050522,0.866208,0.064875,0.571429,0.692308,0.824176,0.857143,0.747253,...,0.449791,0.370654,0.000000,0.0,0.0,0.0,0.416667,0.666667,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,regression,0.822613,0.060846,0.782737,0.052271,0.933333,0.876190,0.971429,0.780952,1.000000,...,0.300795,0.586815,0.263158,0.0,0.0,0.0,0.105263,0.000000,0.000000,0.210526
381,regression,0.822613,0.060846,0.704339,0.077230,0.295238,0.376190,0.457143,0.547619,0.342857,...,0.332165,0.503830,0.052632,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.052632
382,regression,0.822613,0.060846,0.583572,0.091257,0.261905,0.390476,0.423810,0.566667,0.238095,...,0.327960,0.505112,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.052632
383,regression,0.822613,0.060846,0.658487,0.085550,0.295238,0.309524,0.404762,0.490476,0.209524,...,0.316619,0.482002,0.105263,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000


In [11]:
df["scores_ratio"] = df["scores"] / df["original_scores"]
df["scores_std_ratio"] = df["scores_std"] / df["original_scores_std"]

In [12]:
df.to_csv("results/dcc_results.csv", index=False)

In [13]:
df = pd.read_csv("results/dcc_results.csv")
df

,type,original_scores,original_scores_std,scores,scores_std,dcc_pearson,dcc_spearman,dcc_kendall,dcc_mi,dcc_js,...,corr_cnt_good_pearson,corr_cnt_good_spearman,corr_cnt_good_kendall,corr_cnt_good_mi,corr_cnt_good_js,corr_cnt_good_wd,corr_cnt_good_xi,corr_cnt_good_dcor,scores_ratio,scores_std_ratio
0,classification,0.892254,0.050522,0.900000,0.041102,0.978022,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.0,0.0,0.0,0.416667,0.583333,0.000000,0.000000,1.008682,0.813537
1,classification,0.892254,0.050522,0.872085,0.043611,0.560440,0.692308,0.802198,0.857143,0.846154,...,0.000000,0.0,0.0,0.0,0.500000,0.666667,0.000000,0.000000,0.977396,0.863210
2,classification,0.892254,0.050522,0.897085,0.042266,0.582418,0.626374,0.747253,0.791209,0.736264,...,0.000000,0.0,0.0,0.0,0.416667,0.750000,0.083333,0.000000,1.005415,0.836585
3,classification,0.892254,0.050522,0.881350,0.053609,0.472527,0.670330,0.769231,0.890110,0.692308,...,0.000000,0.0,0.0,0.0,0.416667,0.666667,0.000000,0.000000,0.987779,1.061108
4,classification,0.892254,0.050522,0.866208,0.064875,0.571429,0.692308,0.824176,0.857143,0.747253,...,0.000000,0.0,0.0,0.0,0.416667,0.666667,0.000000,0.000000,0.970809,1.284101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,regression,0.822613,0.060846,0.782737,0.052271,0.933333,0.876190,0.971429,0.780952,1.000000,...,0.263158,0.0,0.0,0.0,0.105263,0.000000,0.000000,0.210526,0.951525,0.859059
381,regression,0.822613,0.060846,0.704339,0.077230,0.295238,0.376190,0.457143,0.547619,0.342857,...,0.052632,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.052632,0.856221,1.269259
382,regression,0.822613,0.060846,0.583572,0.091257,0.261905,0.390476,0.423810,0.566667,0.238095,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.052632,0.709413,1.499799
383,regression,0.822613,0.060846,0.658487,0.085550,0.295238,0.309524,0.404762,0.490476,0.209524,...,0.105263,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.800482,1.406006


In [14]:
plot_df = df[df["perturbation_type"] == "Deletion"]
plot_df = plot_df[plot_df["type"] == "regression"]
plot_df = plot_df.select_dtypes(include=[np.number])
fig, ax = plt.subplots(figsize=(32, 32))
sns.heatmap(
    plot_df.corr(), annot=True, fmt=".2f", cmap="coolwarm", vmin=-1, vmax=1, square=True
)

<Axes: >